# 画像分類時の前処理まとめ

## 1. 前処理

### Mean Subtraction

入力画像から平均を引く。各ピクセルから引く。<br>

VGG で採用されている。

https://arxiv.org/pdf/1409.1556.pdf<br>
VGG：2.1 ARCHITECTURE  から抜粋<br>
During training, the input to our ConvNets is a fixed-size 224 × 224 RGB image. <br>
The only preprocessing we do is **subtracting the mean RGB value**, computed on the training set, 
from each pixel.The image is passed through a stack of convolutional (conv.) layers, 
where we use filters with a very small receptive field: 3 × 3 (which is the smallest size to capture the notion of left/right, up/down,center). 

### Per-pixel Mean Subtraction

入力画像から平均を引く。ピクセル・チャンネルごとに計算された平均を引く。<br>
即ち、224x224x3 個の値について個別に平均を計算し用いる。<br>

AlexNet、ResNet で採用されている

https://arxiv.org/pdf/1512.03385.pdf<br>
ResNet：3.4. Implementation から抜粋<br>
A 224×224 crop is randomly sampled from an image or its horizontal flip, <br>
**with the per-pixel mean subtracted** [21]. The standard color augmentation in [21] is used. 

### Random Crop

画像をランダムにトリミングする<br>

AlexNet で採用<br>
- 256x256 ピクセルに画像をリサイズし、そこから 224x224 のパッチをランダムに取り出す

Chainer の ImageNet サンプルはこれと Horizontal Flip をやっている<br>

### Horizontal Flip

画像をランダムに水平方向に反転する。

ImageNet では縦方向はなく水平方向の flip のみが使われる<br>
AlexNet 論文以降必ず使われている模様


## 2. 前処理その2 Augmentation

### Scale Augmentation

まず画像をリサイズする。[256, 480] からランダムに短辺の長さを選ぶ。<br>
次に、224x224 のパッチをそこからランダムサンプルする。

VGG 論文から採用、ResNet 本家論文でも採用

### Aspect Ratio Augmentation

上の Scale Augmentation(画像リサイズ) に加え画像のアスペクト比を [3/4, 4/3] で変換する

GoogLeNet の論文で使われている。

### Color Augmentation

各ピクセルの RGB を 3 次元のベクトルの集合だと考え PCA をかける。<br>
ガウス分布でノイズを生成し、固有ベクトル方向にノイズを加える。<br>
乱数は各ピクセルではなくパッチ全体に対して共通。

AlexNet で採用<br>
論文によると AlexNet の精度への寄与は 1% 程度。<br>
ResNet 本家論文でも使われている。


## 3. テスト時

### Ensemble

複数のモデルを独立に学習し、それらの予測を平均する。

GoogLeNet 論文では、ネットワークは同じだが入力の処理法を変えた 7 つのモデルを<br>
アンサンブルしている。<br>
single crop だと top-5 error が 2% 弱程度向上。

ResNet 本家論文では、34B, 34C, 50, 101, 152×2 の 6 モデルをアンサンブルしている。<br>
fully-convolutional-form で top-5 error に 1% 弱程度向上。

### 10-crop Testing

テスト画像 1 つから data augmentation と類似した手順で 10 個のパッチを切り出し、<br>
それぞれに対する予測を平均して答える。

AlexNet 論文では、（4 スミ＋中央）×（反転の有無）で 10 個のパッチを切り出している。<br>
GoogLeNet は 144 パッチも試してる。<br>
ResNet でも採用している。

GoogLeNet 論文では、crop 数の精度への影響が載っている。<br>
top-5 error で、10 crops で約 1% 弱向上、<br>
144 crops で 2% 強向上（下表は GoogLeNet 論文より引用）。

<img src="https://cdn-ak.f.st-hatena.com/images/fotolife/i/iwiwi/20161231/20161231213228.png"
width="380px" height="380px">

### Fully-convolutional-form Testing

まず、全結合層たちを畳み込み層とみなす。

例えば、直前の画像サイズが s×s であれば、最初の全結合層は s×s → 1×1 の畳み込み層であるとみなす。<br>
すると、ネットワークは全レイヤーが畳み込みとなる。fully convolutional なネットワークの利点は、<br>
入力の画像サイズが変化しても適用することができることである（ただし、出力サイズも変化する）。

テスト画像をリサイズする。この時の画像サイズは、学習で使っている画像サイズと一致しているとは限らないし、
正方形とも限らない。

例えば、短辺を 480 にする。ネットワークをこの画像に適用する。出力を average pooling してそれを予測とする。

さらに、テスト画像のサイズを複数試し、その結果の平均を用いる。

例えば、ResNet 本家論文では短辺を 224, 256, 384, 480, 640 になるようにリサイズしている。<br>
また、horizontal flip も試して平均を取る。

VGG から使われている。<br>
ResNet 本家論文を見るところ、10-crop Testing と比べて、大体 2% 程度精度に寄与している。

## 参考文献

[1] VGG‒Net (2014)<br>
Very Deep Convolutional Networks for Large‒Scale Visual Recognition. arXiv: 1409.1556, 2014. <br>
https://arxiv.org/pdf/1409.1556.pdf

[2]GoogleNet(2014)<br>
Going deeper with convolutions. arXiv: 1409.4842, 2014. <br>
https://arxiv.org/pdf/1409.4842.pdf

[3]ResNet(2015)<br>
Deep Residual Learning for Image Recognition. arXiv:1512.03385, 2015.<br>
https://arxiv.org/pdf/1512.03385.pdf

[4]CNN による画像分類で使われる前処理・テスト時処理まとめ<br>
http://iwiwi.hatenadiary.jp/entry/2016/12/31/162059

[5]画像の水増し方法をTensorFlowのコードから学ぶ<br>
https://qiita.com/Hironsan/items/e20d0c01c95cb2e08b94